In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper

In [3]:
pmw = pymongo_wrapper()

In [4]:
zztable = pmw.get_collection('jiankang39','zz')

In [6]:
zzlist = list(pmw.find_all(zztable,fieldlist=['症状名称','相关疾病']))
zzlist[:2]

[{'症状名称': '脑血管痉挛', '相关疾病': ['颅内动脉瘤', '小儿甲状旁腺功能减退症']},
 {'症状名称': '脑干出血',
  '相关疾病': ['高血压性脑出血', '脑出血', '脑胶质瘤', '小脑脑桥角脑膜瘤', '小儿原发性脑干损伤', '小儿脑干胶质瘤']}]

In [7]:
zzs = list({i['症状名称'] for i in zzlist})
len(zzs)

5898

In [11]:
diseases = set()
for i in zzlist:
    diseases = diseases.union(i['相关疾病'])
diseases = list(diseases)
len(diseases)

7068

In [12]:
from utils.pandas_wrapper import pandas_wrapper as pdw 

In [13]:
zzseries = pdw.build_series(zzs)
zzseries[:5]

0       眼睛刺痛
1        足跟痛
2     口腔黏膜脱落
3     皮下脂肪增多
4    甲状腺功能低下
dtype: object

In [15]:
disseries = pdw.build_series(diseases)
disseries[:5]

0      缩窄性心包炎
1    小儿骨与关节结核
2        巨大胎儿
3      药物性食管炎
4       疱疹样湿疹
dtype: object

In [17]:
zzseries_inv = pdw.build_series(zzseries.index,index=zzseries.values)
zzseries_inv[:5]

眼睛刺痛       0
足跟痛        1
口腔黏膜脱落     2
皮下脂肪增多     3
甲状腺功能低下    4
dtype: int64

In [18]:
disseries_inv = pdw.build_series(disseries.index,index=disseries.values)
disseries_inv[:5]

缩窄性心包炎      0
小儿骨与关节结核    1
巨大胎儿        2
药物性食管炎      3
疱疹样湿疹       4
dtype: int64

In [19]:
from utils.numpy_wrapper import numpy_wrapper as npw

In [20]:
zzdismatr = npw.build_zeros_array((len(zzseries),len(disseries)))

In [21]:
for i in zzlist:
    x = zzseries_inv[i['症状名称']]
    for ii in i['相关疾病']:
        zzdismatr[x,disseries_inv[ii]] = 1

In [26]:
diszzmatr = zzdismatr.T

In [75]:
zz1 = ['脑血管痉挛']
temp = npw.build_zeros_array((1,len(zzseries)))[0]
for i in zz1:
    temp[zzseries_inv[i]] = 1

In [78]:
from math import sqrt
def cal_cos_similarity(a1,a2):
    a = a1.dot(a2)
    b = a1.dot(a1)
    c = a2.dot(a2)
    if b == 0 or c==0:
        return 0
    return a/sqrt( b*c )

In [39]:
diszzmatr[1]

array([0., 0., 0., ..., 0., 0., 0.])

In [41]:
diszzmatr.shape

(7068, 5898)

In [42]:
temp.shape

(1, 5898)

In [116]:
def zz2disease(zzs,n=None):
    temp = npw.build_zeros_array((1,len(zzseries)))[0]
    for i in zzs:
        temp[zzseries_inv[i]] = 1
    l1 = list()
    for i in range(len(disseries)):
        l1.append(cal_cos_similarity(temp,diszzmatr[i]))
    s1 = pdw.build_series(l1)
    s2 = s1.sort_values(ascending=False)
    if n:
        list(disseries[s2.iloc[s2.nonzero()].index].values)[:n],list(s2.iloc[s2.nonzero()].values)[:n]
    return list(disseries[s2.iloc[s2.nonzero()].index].values),list(s2.iloc[s2.nonzero()].values)

In [123]:
zz1 = ['牙龈出血']
zz2disease(zz1)

(['δ-贮存池病',
  '血小板无力症',
  '药物免疫性血小板减少性紫癜',
  '小儿血小板释放功能缺陷性疾患',
  '再生障碍性贫血',
  '严重肝病引起获得性凝血因子异常',
  'α-贮存池病',
  '产科播散性血管内凝血',
  '牙宣',
  '登革出血热',
  '外阴-阴道-牙龈综合征',
  '黑热病',
  '肥大性龈炎',
  '维生素K缺乏病',
  '小儿高黏稠综合征',
  '暴发性肝炎样综合征',
  '小儿巨大血小板综合征',
  '创伤性口炎',
  '小儿血管性血友病',
  '小儿血友病',
  '血小板型假性血管性血友病',
  '真性红细胞增多症',
  '小儿血小板无力症',
  '老年人肝硬化',
  '轻链病与轻链沉积病',
  '小儿维生素C缺乏病',
  '急性失血所致贫血',
  '白血病',
  '汞中毒',
  '血管性血友病',
  '小儿肾性贫血',
  '急性白血病',
  '小儿出血性疾病',
  '小儿继发性免疫性血小板减少性紫癜',
  '小儿特发性血小板减少性紫癜',
  '小儿过敏性紫癜',
  '特发性血小板减少性紫癜',
  '小儿急性髓样白血病',
  '小儿播散性血管内凝血',
  '肝纤维化',
  '坏血病',
  '牙周炎',
  '白血病视网膜病变',
  '牙周病',
  '老年人慢性肾功能衰竭'],
 [0.7071067811865475,
  0.7071067811865475,
  0.5773502691896258,
  0.5,
  0.5,
  0.5,
  0.5,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4472135954999579,
  0.4082482904638631,
  0.4082482904638631,
  0.3779644730092272,
  0.3779644730092272,
  0.3779644730092272,
  0.377

In [117]:
zz1 = ['脑血管痉挛']
zz2disease(zz1)

(['颅内动脉瘤', '小儿甲状旁腺功能减退症'], [0.35355339059327373, 0.2672612419124244])

In [119]:
zz1 = ['恶心','头痛','乏力','健忘']
zz2disease(zz1)

(['蛋白胨性荨麻疹',
  '慢性鼻咽炎',
  '鼻咽炎',
  '功能性呕吐',
  '空调病',
  '胃内异物',
  '贾第虫病',
  '宿醉',
  '药物中毒',
  '胰腺肿瘤',
  '颈部软组织损伤',
  '头风病',
  '溢饮',
  '腮腺炎',
  '中性粒细胞减少症',
  '慢性粒细胞白血病',
  '急性无黄疸型肝炎',
  '阑尾炎',
  '中老年男子部分性雄激素缺乏综合征',
  '原发性中枢神经系统淋巴瘤',
  '寒冷性荨麻疹',
  '老年人血管性痴呆',
  '房事头疼症',
  '上颌窦炎',
  '特发性室性心动过速',
  '原发性高血压',
  '肝阳上亢',
  '肾小管间质性肾炎葡萄膜炎综合征',
  '原发性下肢深静脉瓣膜关闭不全',
  '电脑视觉综合症',
  '溃疡病出血',
  '风温',
  '慢性荨麻疹',
  '密码综合症',
  '慢性白血病',
  '职业枯竭',
  '寒湿泄泻',
  'Castleman病',
  '夏季低热',
  '糖尿病并结核病',
  '肾精亏虚',
  '旅途精神病',
  '急性上颌窦炎',
  '葡萄球菌感染',
  '胆热',
  '慢性乳腺炎',
  '急性糜烂性胃炎',
  '自身红细胞过敏性紫癜',
  '小儿流涎症',
  '慢性乙肝',
  '性功能异常伴发的精神障碍',
  '女子梦交',
  '阴挺',
  '老年人高脂血症',
  '性病相关性胃肠道感染',
  '阳明头痛',
  '电视眼病',
  '心虚',
  '休息痢',
  '一氧化碳中毒',
  '小儿暑温',
  '红细胞增多症',
  '生长痛',
  '腔隙性脑梗死',
  '淋巴细胞白血病',
  '肝热病',
  '风热头痛',
  '疫毒痢',
  '性交晕厥',
  '病毒性支气管炎',
  '糙皮病',
  '传染性淋巴细胞增多症',
  '镉中毒',
  '老年痴呆症',
  '淋病后综合征',
  '虚寒咳嗽',
  '瘀血头痛',
  '原发性骨髓纤维化',
  '血虚头痛',
  '脑萎',
  '肺螨病',
  '成人T细胞白血病',
  '杨梅疮',
  '老年人低血糖昏迷',
  '空肠、回肠憩室',
  '急性细菌性前列腺炎',
  

In [120]:
zz1 = ['小腿酸痛','小腿发胀','脚后跟疼']
zz2disease(zz1)

(['情感性交叉擦腿症',
  '原发性下肢静脉瓣膜关闭不全',
  '血栓闭塞性脉管炎',
  '腰腿痛',
  '老年静脉血栓症',
  '下肢静脉血栓',
  '急性单纯外伤性腰腿痛',
  '慢性单纯外伤性腰腿痛'],
 [0.2886751345948129,
  0.2581988897471611,
  0.2182178902359924,
  0.19245008972987526,
  0.16666666666666666,
  0.16666666666666666,
  0.1543033499620919,
  0.14907119849998599])

In [124]:
distable = pmw.get_collection('jiankang39','diseases')

In [150]:
def zz2ks(zzs):
    a,b = zz2disease(zzs)
    ksdict = dict()
    for i in range(len(a)):
        ks = distable.find_one({'疾病名称':a[i]})
        if ks and '挂号的科室' in ks:
            ks = ks['挂号的科室'].split()
            print(a[i],ks)
            for ii in ks:
                if ii in ksdict:
                    ksdict[ii] += b[i]
                else:
                    ksdict[ii] = b[i]
    return pdw.build_series(ksdict).sort_values(ascending=False)

In [96]:
list(disseries[s2.iloc[s2.nonzero()].index].values)

['颅内动脉瘤', '小儿甲状旁腺功能减退症']

In [147]:
zz2ks(['小腿酸痛','小腿发胀','脚后跟疼'])

血管外科    0.809750
老年科     0.359117
骨科      0.303375
儿科      0.288675
普外科     0.192450
外伤科     0.154303
dtype: float64

In [151]:
zz2ks(['恶心','头痛','乏力','健忘'])

蛋白胨性荨麻疹 ['皮肤科']
慢性鼻咽炎 ['耳鼻喉科']
鼻咽炎 ['耳鼻喉科']
功能性呕吐 ['消化内科', '内科']
胃内异物 ['消化内科']
贾第虫病 ['消化内科', '传染科']
药物中毒 ['急诊科', '消化内科']
胰腺肿瘤 ['肿瘤科', '肝胆外科']
头风病 ['中医科']
溢饮 ['中医科']
腮腺炎 ['内分泌科', '口腔科']
中性粒细胞减少症 ['血液科']
慢性粒细胞白血病 ['血液科']
急性无黄疸型肝炎 ['消化内科', '传染科']
阑尾炎 ['消化内科', '胃肠外科']
中老年男子部分性雄激素缺乏综合征 ['内分泌科', '男科']
原发性中枢神经系统淋巴瘤 ['脑外科', '肿瘤科']
寒冷性荨麻疹 ['皮肤科']
老年人血管性痴呆 ['神经内科', '老年科']
房事头疼症 ['中医科']
上颌窦炎 ['耳鼻喉科']
特发性室性心动过速 ['心血管内科']
原发性高血压 ['心血管内科']
肝阳上亢 ['中医科']
肾小管间质性肾炎葡萄膜炎综合征 ['眼科', '肾内科']
原发性下肢深静脉瓣膜关闭不全 ['血管外科']
电脑视觉综合症 ['眼科', '营养科']
溃疡病出血 ['消化内科', '胃肠外科']
风温 ['中医科']
慢性荨麻疹 ['皮肤科']
密码综合症 ['心理咨询']
慢性白血病 ['血液科']
职业枯竭 ['心理咨询']
寒湿泄泻 ['消化内科', '中医科']
Castleman病 ['血液科', '内分泌科']
糖尿病并结核病 ['内分泌科', '传染科']
肾精亏虚 ['中医科']
旅途精神病 ['心理咨询', '精神病科']
急性上颌窦炎 ['耳鼻喉科']
葡萄球菌感染 ['内科', '传染科']
胆热 ['中医科', '消化内科']
慢性乳腺炎 ['妇科']
急性糜烂性胃炎 ['消化内科']
自身红细胞过敏性紫癜 ['血液科']
小儿流涎症 ['中医科']
慢性乙肝 ['肝炎', '消化内科']
性功能异常伴发的精神障碍 ['男科', '心理咨询']
女子梦交 ['中医科']
阴挺 ['中医科']
老年人高脂血症 ['心血管内科', '老年科']
性病相关性胃肠道感染 ['消化内科', '传染科']
阳明头痛 ['中医科']
电视眼病 ['眼科']
心虚 ['中医科']
休息痢

中医科      8.403217
消化内科     6.259193
血液科      3.090876
传染科      2.194509
神经内科     1.804866
内分泌科     1.786424
心理咨询     1.771219
肿瘤科      1.769166
心血管内科    1.728011
呼吸内科     1.664396
耳鼻喉科     1.542612
皮肤科      1.533716
眼科       1.259112
急诊科      1.175211
内科       1.163038
肝胆外科     1.108565
性病科      0.853501
男科       0.836243
老年科      0.798675
胃肠外科     0.712433
妇科       0.642878
脑外科      0.616272
口腔科      0.482653
营养科      0.438982
肾内科      0.424866
血管外科     0.371268
精神病科     0.335410
肝炎       0.324818
儿科       0.318082
普外科      0.308870
心胸外科     0.239144
体检保健科    0.214849
肝病       0.204124
泌尿外科     0.176777
肛肠外科     0.158114
产科       0.129099
烧伤科      0.121268
风湿科      0.114708
dtype: float64

In [152]:
zz2ks(['牙龈出血'])

δ-贮存池病 ['血液科']
血小板无力症 ['血液科']
药物免疫性血小板减少性紫癜 ['血液科']
小儿血小板释放功能缺陷性疾患 ['儿科', '血液科']
再生障碍性贫血 ['血液科']
严重肝病引起获得性凝血因子异常 ['血液科']
α-贮存池病 ['血液科']
产科播散性血管内凝血 ['产科', '急诊科']
牙宣 ['口腔科', '中医科']
登革出血热 ['传染科', '急诊科']
外阴-阴道-牙龈综合征 ['妇科', '口腔科']
黑热病 ['传染科']
肥大性龈炎 ['口腔科']
维生素K缺乏病 ['血液科', '内科']
小儿高黏稠综合征 ['儿科', '血液科']
暴发性肝炎样综合征 ['肝炎', '传染科']
小儿巨大血小板综合征 ['儿科', '血液科']
创伤性口炎 ['口腔科', '外伤科']
小儿血管性血友病 ['儿科', '血液科']
小儿血友病 ['儿科', '血液科']
血小板型假性血管性血友病 ['血液科']
真性红细胞增多症 ['血液科']
小儿血小板无力症 ['儿科', '血液科']
老年人肝硬化 ['老年科', '肝胆外科']
轻链病与轻链沉积病 ['血液科', '肾内科']
小儿维生素C缺乏病 ['儿科', '营养科']
急性失血所致贫血 ['血液科', '内科']
白血病 ['血液科', '肿瘤科']
汞中毒 ['急诊科']
血管性血友病 ['血液科']
小儿肾性贫血 ['儿科', '血液科']
急性白血病 ['血液科']
小儿出血性疾病 ['儿科', '血液科']
小儿继发性免疫性血小板减少性紫癜 ['儿科', '血液科']
小儿特发性血小板减少性紫癜 ['儿科', '血液科']
小儿过敏性紫癜 ['儿科', '消化内科']
特发性血小板减少性紫癜 ['血液科', '内科']
小儿急性髓样白血病 ['儿科', '血液科']
小儿播散性血管内凝血 ['儿科', '血液科']
肝纤维化 ['肝病', '消化内科']
坏血病 ['儿科', '营养科']
牙周炎 ['口腔科']
白血病视网膜病变 ['眼科', '血液科']
牙周病 ['口腔科']
老年人慢性肾功能衰竭 ['肾内科', '老年科']


血液科     11.086158
儿科       5.157046
口腔科      2.136272
传染科      1.302675
急诊科      1.227761
内科       1.069222
营养科      0.603553
消化内科     0.566025
肾内科      0.499418
老年科      0.499418
中医科      0.447214
产科       0.447214
妇科       0.447214
肝炎       0.408248
外伤科      0.377964
肝胆外科     0.353553
肿瘤科      0.333333
肝病       0.277350
眼科       0.200000
dtype: float64